In [1]:
#import support lib
import json
import os
from PIL import Image, ImageOps
import pydicom
import numpy as np
import shutil
from collections import OrderedDict
import cv2
from threading import Thread
import math
from random import shuffle
import _thread
import time

In [2]:
# import local lib
from Preprocess.PreprocessImage import PreprocessImage
from Database.Sqlite3 import DB_SQLITE3
from Preprocess.SupportLibrary import SupportLibrary
from Preprocess.AugmentationImage import AugmentationImage

In [3]:
# config augment
# augmentation = True
# augmentation_list = ["rotation", "horizontal_flip", "vertical_flip", "h_flip_rotation", "v_flip_rotation"]
# augmentation_all_class = True
nb_rotation = 1

# config preprocess
# image_size = [224, 224]
image_size = [256, 256]
inner_preprocess = True
# preprocess_list = ["RGB", "clahe_r", "resize"]
preprocess_list = ["RGB",  "resize"]

array_preprocess = True
array_preprocess_input_list = [
        ["01normalization"],
        ["01normalization", "log"]
]

In [16]:
with open('resource/state/state_1031_none.json', 'r') as json_file:  
    state_file = json.load(json_file)

db_img = DB_SQLITE3("resource/db/database_1031_img_none.db")
db_label = DB_SQLITE3("resource/db/database_1031_label_none.db")

## preprocess using thread

In [5]:
count = 0
# lock = _thread.allocate_lock()
maximum_thread = 20
current_thread = 0

In [6]:
# augmentator = AugmentationImage(augmentation_list)
# nb_augmentation = augmentator.calculate_augmentation_part()
pre_img = PreprocessImage()

In [7]:
def preprocess_augment(x, current_augmentation, save_path):
    
    global current_thread
    current_thread+=1
    
    try:
         # get img path to open
        path = x[0][1].replace('dataset_ev', 'dataset_ev')
        #print(img_id, path)

        img = Image.open(path)
        # augmentation
        img = augmentator.data_augmentation(img, current_augmentation)
        # pro-process
        for preprocess_method in preprocess_list:
            img = pre_img.preprocess(img, preprocess_method)

        if current_augmentation == 0:
            img_name = x[0][4] + '.png'
        else:
            img_name = x[0][4] + '_' + str(current_augmentation) + '.png'

        img = img.save(save_path + img_name)
        
    finally:
        current_thread-=1

In [18]:
def save_folder(x, current_augmentation, save_path ):
    
    global current_thread
    current_thread+=1
    
    try:
         # get img path to open
#         print(x)
        path = x[0][1].replace('dataset_ev', 'dataset_ev')
        #print(img_id, path)

        img = Image.open(path)

        # pro-process
        for preprocess_method in preprocess_list:
            img = pre_img.preprocess(img, preprocess_method)

        if current_augmentation == 0:
            img_name = x[0][4] + '.png'
        else:
            img_name = x[0][4] + '_' + str(current_augmentation) + '.png'

        img = img.save(save_path + img_name)
        
    finally:
        current_thread-=1

In [9]:
# augmentation image in each 5-fold cross validation
for index, fold in enumerate(state_file['cross_validation_id']):
    # print(fold, end="\n\n")
#     print(fold)
    fold_number = str(index)
#     print("fold: {}, number of augmentation: {}".format(index, nb_augmentation))

    save_path = 'Dataset/Fold_img_unet1/' + fold_number + '/'
#     save_path = 'resource/normal_clahe_r/'

#     if os.path.exists(save_path):
#         shutil.rmtree(save_path)

    if not os.path.exists(save_path):
        os.makedirs(save_path)

#     for current_augmentation in range(0,nb_augmentation): 
        #print('current_augmentation: ', current_augmentation)
    current_augmentation=0
    for img_id in fold:
        x = db_img.select_by_cond(table_name='dataset_info', dict_condition = {'id':str(img_id)}, list_condition_operation = ['='])

        while current_thread > maximum_thread:
            time.sleep(0.5)
        _thread.start_new_thread(save_folder,(x, current_augmentation, save_path))
        count+=1

In [19]:
# augmentation image in each 5-fold cross validation
for index, img_id in enumerate(state_file['cross_validation_test_id']):
    # print(fold, end="\n\n")
#     print(fold)
#     fold_number = str(index)
#     print("fold: {}, number of augmentation: {}".format(index, nb_augmentation))

    save_path = 'Dataset/Unet1_v2/Test/label/'

#     if os.path.exists(save_path):
#         shutil.rmtree(save_path)

    if not os.path.exists(save_path):
        os.makedirs(save_path)

#     for current_augmentation in range(0,nb_augmentation): 
        #print('current_augmentation: ', current_augmentation)
    current_augmentation=0
#     for img_id in fold:
    x = db_label.select_by_cond(table_name='dataset_info', dict_condition = {'id':str(img_id)}, list_condition_operation = ['='])
    
#     save_folder(x, current_augmentation, save_path)
    
    while current_thread > maximum_thread:
        time.sleep(0.5)
    _thread.start_new_thread(save_folder,(x, current_augmentation, save_path))
    count+=1

In [ ]:
from glob import glob
len(glob('Dataset/preprocess_removebg/0/*'))

#### preprocess and augment

In [ ]:
# augmentation image in each 5-fold cross validation
for index, fold in enumerate(state_file['cross_validation_id']):
    # print(fold, end="\n\n")
    fold_number = str(index)
    
    augmentator = AugmentationImage(augmentation_list)
    nb_augmentation = augmentator.calculate_augmentation_part()
    print("fold: {}, number of augmentation: {}".format(index, nb_augmentation))
    
    pre_img = PreprocessImage()
    
    save_path = 'Dataset/preprocess_original/' + fold_number + '/'
    
    if os.path.exists(save_path):
        shutil.rmtree(save_path)
            
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    for current_augmentation in range(0,nb_augmentation): 
        print('current_augmentation: ', current_augmentation)
        for img_id in fold:
            x = db.select_by_cond(table_name='dataset_info', dict_condition = {'id':str(img_id)}, list_condition_operation = ['='])

            # get img path to open
            path = x[0][1].replace('dataset_ev', 'dataset')
            #print(img_id, path)
            img = Image.open(path)
            # augmentation
            img = augmentator.data_augmentation(img, current_augmentation)
            # pro-process
            for preprocess_method in preprocess_list:
                img = pre_img.preprocess(img, preprocess_method)
              
            if current_augmentation == 0:
                img_name = x[0][4] + '.png'
            else:
                img_name = x[0][4] + '_' + str(current_augmentation) + '.png'
            
            img = img.save(save_path + img_name)

fold: 0, number of augmentation: 1080
current_augmentation:  0
current_augmentation:  1
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
rotation
current_augmentation:  2
current_augmentation:  3
current_augmentation:  4
current_augmentation:  5
current_augmentation:  6
current_augmentation:  7
current_augmentation:  8
current_augmentation:  9
current_augmentation:  10
current_augmentation:  11
current_augm

In [12]:
nb_augmentation

1080

In [13]:
# update db of 5-fold set
import os

count_id = 407

for index, fold in enumerate(state_file['cross_validation_id']):
    
    fold_number = str(index)
    
    for img_id in fold:
        
        x = db_img.select_by_cond(table_name='dataset_info', dict_condition = {'id':str(img_id)}, list_condition_operation = ['='])
        
        # new path
        img_path = 'Dataset/preprocess_unet1_410/' + fold_number + '/' + x[0][4] + '.png'
        # rename image
        # os.rename(img_path.replace(x[0][4], x[0][4] + '_0'), img_path) 
        # update path of original images
        newdb.update_by_id("dataset_info", img_id, {'path':img_path})
        
        ## insert augmentation images to db
        for current_augmentation in range(1, nb_augmentation):  
            
            img_path = 'Dataset/preprocess_unet1_410/' + fold_number + '/' + x[0][4] + '_' + str(current_augmentation) + '.png'
                
            count_id += 1
                
            data_dict = {'id': str(count_id), 'path': img_path,'type': x[0][2],'using_type':'train'}
            newdb.insert('dataset_info', data_dict)

In [14]:
# update path of test set
for img_id in state_file['cross_validation_test_id']:
    x = db.select_by_cond(table_name='dataset_info', dict_condition = {'id':str(img_id)}, list_condition_operation = ['='])
    path = x[0][1]
    newdb.update_by_id("dataset_info", img_id,{'path': path.replace('dataset_ev', 'unet1/dataset_preprocess_bg')})

In [15]:
newdb.commit()

In [10]:
json_file.close()

In [ ]:
from random import shuffle

In [ ]:
# edit state.json
end = 408
for index, fold in enumerate(state_file['cross_validation_id']):
    print(fold, end="\n\n")
    new_fold = []
    new_fold.extend(fold)
    
    for img_id in fold:
        for i in range(1, nb_augmentation):
            new_fold.append(end)
            end += 1
            
    print(len(fold), end='\n\n')
    
    shuffle(new_fold)
    print(new_fold)
    state_file['cross_validation_id'][index] = new_fold
    with open('resource/state/state.json', 'w') as json_file:  
        json.dump(state_file, json_file)
    json_file.close()